In [7]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Завантаження даних
df = pd.read_csv("ai_adoption.csv")

# 1. Мапа інвестицій по країнах (останній рік)
latest_year = df["Year"].max()
latest_df = df[df["Year"] == latest_year]

fig_map = px.choropleth(
    latest_df,
    locations="Country",
    locationmode="country names",
    color="Investment_Mln",
    hover_name="Country",
    title=f"AI investments by country in {latest_year}",
    color_continuous_scale="PuBu"
)
fig_map.update_geos(showcoastlines=True, projection_type="natural earth")
fig_map.update_layout(margin=dict(l=20, r=20, t=60, b=20), title_x=0.5, height=600)
fig_map.show()


# 2. Комбінований графік: AI Projects + Investment
agg = df.groupby("Year").agg({"AI_Projects": "sum", "Investment_Mln": "sum"}).reset_index()
# Комбінований графік
fig_combo = go.Figure()
# Стовпчики: AI Projects
fig_combo.add_trace(go.Bar(
    x=agg["Year"],
    y=agg["AI_Projects"],
    name="AI Projects",
    yaxis="y1",
    text=agg["AI_Projects"],
    textposition="outside",
    marker=dict(color=agg["AI_Projects"], colorscale="Blues", showscale=False),
    hovertemplate="Year: %{x}<br>Projects: %{y}<extra></extra>"
))
# Лінія: Investment ($M)
fig_combo.add_trace(go.Scatter(
    x=agg["Year"],
    y=agg["Investment_Mln"],
    name="Investment ($M)",
    yaxis="y2",
    mode="lines+markers+text",
    line=dict(color="#5e3c99", width=3), 
    textposition="top center",
    hovertemplate="Year: %{x}<br>Investments: %{y}<extra></extra>"
))

fig_combo.update_layout(
    title="AI Projects and Investments by year",
    xaxis_title="Year",
    yaxis=dict(title="Number of projects", side="left", showgrid=True),
    yaxis2=dict(title="Investments ($M)", overlaying="y", side="right", showgrid=False),
    legend=dict(x=0.01, y=0.99),
    plot_bgcolor="white",   
    paper_bgcolor="white", 
    margin=dict(l=40, r=40, t=60, b=40)
)

fig_combo.show()


# 3. Анімована динаміка (bubble chart)
color_map = { "Australia": "#6A5ACD", "Brazil": "#7B68EE", "Canada": "#483D8B", "China": "#9370DB", "France": "#8A2BE2", "Germany": "#4169E1", 
              "India": "#6495ED", "Japan": "#1E90FF", "United Kingdom": "#0050EF", "USA": "#4B0082" }

fig_bubble = px.scatter(
    df,
    x="Investment_Mln",
    y="AI_Users_Mln",
    size="AI_Projects",
    color="Country",
    animation_frame="Year",
    animation_group="Country",
    hover_name="Country",
    size_max=60,
    title="Dynamics of AI usage by country (2020–2024)",
    color_discrete_map=color_map
)
fig_bubble.update_traces(opacity=0.85, marker=dict(line=dict(width=0.6, color='rgba(0,0,0,0.2)')))
fig_bubble.update_layout(
    xaxis_title="Investments ($M)",
    yaxis_title="AI users (million)",
    template="plotly_white",
    paper_bgcolor="white",
    plot_bgcolor="white",
    margin=dict(l=40, r=40, t=60, b=60),
    font=dict(family="Arial", size=12),
    legend_title="Country"
)
fig_bubble.show()


# 4. Таблиця топ країн
table_df = df.groupby("Country").agg({ "Investment_Mln": "sum", "AI_Projects": "sum", "AI_Users_Mln": "mean" }).reset_index().round(2).sort_values(by="Investment_Mln", ascending=False)

fig_table = go.Figure(data=[
    go.Table(
        header=dict(
            values=["<b>Country</b>", "<b>Investment ($M)</b>", "<b>AI Projects</b>", "<b>Avg AI Users (mln)</b>"],
            fill_color="#5e3c99",
            font=dict(color="white", size=12),
            align="center",
            height=32
        ),
        cells=dict(
            values=[table_df["Country"], table_df["Investment_Mln"], table_df["AI_Projects"], table_df["AI_Users_Mln"]],
            fill_color="rgb(250,250,255)",
            align="center",
            height=28,
            font=dict(size=11)
        )
    )
])
fig_table.update_layout(
    title="Top Countries by AI Investment (2014–2024)",
    title_x=0.5,
    margin=dict(l=10, r=10, t=60, b=20),
    height=480,
    paper_bgcolor="white"
)
fig_table.show()